In [1]:
import backtrader as bt
import alpaca_backtrader_api as api
import alpaca_trade_api as tradeapi
import pandas as pd

from config import key_id, secret_key



In [2]:

from typing import Dict, List
from functools import reduce
from pandas import DataFrame
# --------------------------------
import talib.abstract as tab
from typing import Dict, List
from pandas import DataFrame, DatetimeIndex, merge
# --------------------------------
import talib as ta
import numpy  # noqa


In [ ]:
   def populate_buy_trend(self, dataframe: DataFrame, metadata: dict) -> DataFrame:
        dataframe.loc[
            (
                (dataframe['open'] < dataframe['ema_low']) &
                (dataframe['adx'] > 30) &
                (
                    (dataframe['fastk'] < 30) &
                    (dataframe['fastd'] < 30) &
                    (qtpylib.crossed_above(dataframe['fastk'], dataframe['fastd']))
                )
            ),
            'buy'] = 1
        return dataframe
    def populate_sell_trend(self, dataframe: DataFrame, metadata: dict) -> DataFrame:
        dataframe.loc[
            (
                (dataframe['open'] >= dataframe['ema_high'])
            ) |
            (
                (qtpylib.crossed_above(dataframe['fastk'], 70)) |
                (qtpylib.crossed_above(dataframe['fastd'], 70))
            ),
            'sell'] = 1
        return dataframe

In [3]:
api = tradeapi.REST(key_id, secret_key)

In [4]:
#get data and turn to dataframe form
#15Min, 5Min
data=api.get_barset(symbols = ("INTC"), start = '2020-03-27T11:52:00-04:00', end = "2020-01-01", timeframe = "minute", limit=1000)
df=data.df #converts to dataframe

In [5]:
#get rid of multiindex, otherwise backtrader will have problems
df=df.INTC

In [6]:
#previewing
df.head()

,open,high,low,close,volume
time,,,,,
2020-03-31 12:13:00-04:00,54.555,54.560,54.50,54.500,5928
2020-03-31 12:14:00-04:00,54.505,54.505,54.47,54.500,10618
2020-03-31 12:15:00-04:00,54.500,54.520,54.48,54.485,6958
2020-03-31 12:16:00-04:00,54.490,54.490,54.40,54.400,7964
2020-03-31 12:17:00-04:00,54.400,54.460,54.40,54.460,6858


In [7]:
#feed dataframe to cerbro
datacere = bt.feeds.PandasData(dataname=df)

datacere

In [8]:
#create backtrader
cerebro = bt.Cerebro()

In [9]:
from pandas import DataFrame
def populateindicators(dataframe) -> DataFrame:
        dataframe['ema_high'] = tab.EMA(dataframe, timeperiod=5, price='high')
        dataframe['ema_close'] = tab.EMA(dataframe, timeperiod=5, price='close')
        dataframe['ema_low'] = tab.EMA(dataframe, timeperiod=5, price='low')
        stoch_fast = tab.STOCHF(dataframe, 10.0, 3.0, 0.0, 3.0, 0.0)
        dataframe['fastd'] = stoch_fast['fastd']
        dataframe['fastk'] = stoch_fast['fastk']
        dataframe['adx'] = tab.ADX(dataframe)

        # required for graphing
        bollinger =ta.BBANDS(data.close,timeperiod=10)
        dataframe['bb_lowerband'] = bollinger[2]
        dataframe['bb_upperband'] = bollinger[0]
        dataframe['bb_middleband'] = bollinger[1]

        return dataframe

In [46]:
#feed strat to cerebro


class SmaCross(bt.Strategy):
    # list of parameters which are configurable for the strategy
    params = dict(
        pfast=10,  # period for the fast moving average
        pslow=30   # period for the slow moving average
    )

    def __init__(self):
        sma1 = bt.ind.SMA(period=self.p.pfast)  # fast moving average
        sma2 = bt.ind.SMA(period=self.p.pslow)  # slow moving average
        self.crossover = bt.ind.CrossOver(sma1, sma2)  # crossover signal

    def next(self):
        if not self.position:  # not in the market
            if self.crossover > 0:  # if fast crosses slow to the upside
                self.buy()  # enter long

        elif self.crossover < 0:  # in the market & cross to the downside
            self.close()  # close long position

            
class secondStrategy(bt.Strategy):
#RSI is relative strength index
    def __init__(self):
        self.rsi = bt.indicators.RSI_SMA(self.data.close, period=21)

    def next(self):
        if not self.position:
            if self.rsi < 30:
                self.buy(size=100)
        else:
            if self.rsi > 70:
                self.sell(size=100)

In [47]:
#for right now can only run one strat at a time

#cerebro.addstrategy(SmaCross)
cerebro.addstrategy(secondStrategy)


1

In [12]:
#Add slippage
cerebro.broker = bt.brokers.BackBroker(slip_perc=0.005) 

In [13]:
#set up cerebro

cerebro.broker.setcash(100000)

cerebro.broker.setcommission(commission=0.0)
cerebro.addsizer(bt.sizers.PercentSizer, percents=20)
cerebro.adddata(datacere)



In [14]:
#create plotinfo dict for plotting options
plotinfo = dict(plot=True,
                subplot=True,
                plotname='',
                plotskip=False,
                plotabove=False,
                plotlinelabels=False,
                plotlinevalues=True,
                plotvaluetags=True,
                plotymargin=0.0,
                plotyhlines=[],
                plotyticks=[],
                plothlines=[],
                plotforce=False,
                plotmaster=None,
                plotylimited=True,
           )

In [15]:
#change window settings
import matplotlib.pylab as pylab
pylab.rcParams['figure.figsize'] = 20, 15  # that's default image size for this interactive session
pylab.rcParams['font.family'] = 'sans-serif'
pylab.rcParams['font.sans-serif'] = ['Bitstream Vera Sans']
pylab.rcParams['font.serif'] = ['Bitstream Vera Sans']
pylab.rcParams["font.size"] = "20"

In [16]:
#Add indicators 

#adding another simple moving average line; this is the backtrader built in indicator
sma = bt.indicators.SimpleMovingAverage(datacere, period=15)
sma.plotinfo.plotname = 'mysma'


macd= bt.indicators.MACD(datacere)
macd.plotinfo.plotname="MACD"
#https://www.backtrader.com/docu/indautoref/
#custom: https://medium.com/@danjrod/custom-indicator-development-in-python-with-backtrader-bc775552dc3e

In [17]:
macd.lines.macd

In [18]:
# Add the analyzers we are interested in
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="ta")
cerebro.addanalyzer(bt.analyzers.SQN, _name="sqn")
cerebro.addanalyzer(bt.analyzers.Returns, _name="ret")
cerebro.addanalyzer(bt.analyzers.PeriodStats, _name="pstat")
cerebro.addanalyzer(bt.analyzers.Transactions, _name="trans")



#analyzers list and also creating own analyzer
#https://www.backtrader.com/docu/analyzers/analyzers/
#https://www.backtrader.com/docu/analyzers-reference/

In [19]:
# Add the observers we are interested in 
#this adds more things to the plot on top of the stocks chart
cerebro.addobserver(bt.observers.Broker)
cerebro.addobserver(bt.observers.BuySell)


#https://www.backtrader.com/docu/observers-and-statistics/observers-and-statistics/

In [20]:
# Run over everything
test=cerebro.run()

# Plot the result
cerebro.plot()


TypeError: an integer is required

## Analyzers


In [38]:
# print the SQN analyzers  SystemQualityNumber
#test[0] is the first strategy used. For multiple strategies, use corresponding number in the list.
SQN_dict=test[0].analyzers.sqn.get_analysis() 
tradeanalysis_dict=test[0].analyzers.ta.get_analysis()
ret_dict=test[0].analyzers.ret.get_analysis()
trans_dict=test[0].analyzers.trans.get_analysis()

#.ta.get_analysis()
    #printSQN(firstStrat.analyzers.sqn.get_analysis())

    
#The Backtrader documentation provides a helpful ranking system for SQN:
#SQN measures the relationship between the mean (expectancy) and the standard deviation of the R-multiple distribution 
#generated by a trading system. It also makes an adjustment for the number of trades involved.
#    0 and under is BAD
#    1.6 – 1.9 Below average
#    2.0 – 2.4 Average
#    2.5 – 2.9 Good
#    3.0 – 5.0 Excellent
#    5.1 – 6.9 Superb
#    7.0 – Holy Grail?


In [39]:
SQN_dict

AutoOrderedDict([('sqn', -1.5859082690334405), ('trades', 4)])

In [40]:
ret_dict #in percentages

OrderedDict([('rtot', -0.001506133650071651),
             ('ravg', -0.0005020445500238836),
             ('rnorm', -0.11883926764594406),
             ('rnorm100', -11.883926764594406)])

In [41]:
tradeanalysis_dict
#pnl stands for profit and loss

AutoOrderedDict([('total',
                  AutoOrderedDict([('total', 4), ('open', 0), ('closed', 4)])),
                 ('streak',
                  AutoOrderedDict([('won',
                                    AutoOrderedDict([('current', 1),
                                                     ('longest', 1)])),
                                   ('lost',
                                    AutoOrderedDict([('current', 0),
                                                     ('longest', 3)]))])),
                 ('pnl',
                  AutoOrderedDict([('gross',
                                    AutoOrderedDict([('total',
                                                      -150.50000000000026),
                                                     ('average',
                                                      -37.625000000000064)])),
                                   ('net',
                                    AutoOrderedDict([('total',
                                  

In [23]:
trans_dict


OrderedDict([(datetime.datetime(2020, 3, 31, 16, 35),
              [[100, 53.72, 0, '', -5372.0]]),
             (datetime.datetime(2020, 4, 1, 14, 23),
              [[-100, 53.48, 0, '', 5348.0]]),
             (datetime.datetime(2020, 4, 1, 15, 28),
              [[100, 53.92, 0, '', -5392.0]]),
             (datetime.datetime(2020, 4, 1, 17, 22),
              [[-100, 52.92, 0, '', 5292.0]]),
             (datetime.datetime(2020, 4, 1, 18, 28),
              [[100, 52.73, 0, '', -5273.0]]),
             (datetime.datetime(2020, 4, 2, 14, 8),
              [[-100, 52.165, 0, '', 5216.5]]),
             (datetime.datetime(2020, 4, 2, 17, 15),
              [[100, 53.52, 0, '', -5352.0]]),
             (datetime.datetime(2020, 4, 2, 19, 14),
              [[-100, 53.82, 0, '', 5382.0]])])

In [24]:
trans_df = pd.DataFrame(trans_dict)
trans_df=trans_df.transpose()
trans_df.head()

#'date', 'amount', 'price', 'sid'(0), 'symbol'(empty), 'value'

,0
2020-03-31 16:35:00,"[100, 53.72, 0, , -5372.0]"
2020-04-01 14:23:00,"[-100, 53.48, 0, , 5348.0]"
2020-04-01 15:28:00,"[100, 53.92, 0, , -5392.0]"
2020-04-01 17:22:00,"[-100, 52.92, 0, , 5292.0]"
2020-04-01 18:28:00,"[100, 52.73, 0, , -5273.0]"


In [25]:
trans_df[0][0:][0]

[100, 53.72, 0, '', -5372.0]

## Indicators

In [26]:
#to see the data, will have to use ta-lib
import talib as ta
import pandas as pd

analysis = pd.DataFrame(index = df.index)
timestamps = analysis.index

analysis['ADX'] = ta.ADX(df.high, df.low, df.close)
analysis['RSI'] = ta.RSI(df.close)
analysis['SMA5'], analysis['SMA10'], analysis['SMA20'] = ta.SMA(df.close,5), ta.MA(df.close,10), ta.MA(df.close,20)
analysis['PLUS_DI'] = ta.PLUS_DM(df.high, df.low)
analysis['MINUS_DI'] = ta.MINUS_DM(df.high, df.low)
analysis['BBANDUP'], analysis['BBANDDOWN'] = (ta.BBANDS(df.close,20,2,1.5))[0], (ta.BBANDS(df.close,20,2,1.5))[1]
analysis['%K'], analysis['%D'] = ta.STOCHF(df.high, df.low, df.close)
MACD = ta.MACD(df.close, fastperiod=12, slowperiod=26, signalperiod=9)

In [27]:
analysis.iloc[30:60]

,ADX,RSI,SMA5,SMA10,SMA20,PLUS_DI,MINUS_DI,BBANDUP,BBANDDOWN,%K,%D
time,,,,,,,,,,,
2020-03-31 12:43:00-04:00,75.068781,42.956593,53.814,53.7425,53.82750,0.293194,0.525766,54.145124,53.82750,80.392157,86.928105
2020-03-31 12:44:00-04:00,71.116610,46.679830,53.876,53.7595,53.81150,0.327252,0.488211,54.073838,53.81150,91.176471,83.986928
2020-03-31 12:45:00-04:00,67.446738,45.987155,53.883,53.7785,53.80150,0.303877,0.453339,54.033321,53.80150,67.857143,79.808590
2020-03-31 12:46:00-04:00,63.689016,44.334054,53.869,53.8025,53.79600,0.312171,0.420957,54.016513,53.79600,42.857143,67.296919
2020-03-31 12:47:00-04:00,60.080954,46.973339,53.882,53.8210,53.80100,0.299873,0.390889,54.026579,53.80100,70.000000,60.238095
2020-03-31 12:48:00-04:00,56.606512,47.860750,53.898,53.8560,53.80300,0.288454,0.362968,54.032028,53.80300,75.000000,62.619048
2020-03-31 12:49:00-04:00,53.679153,48.783795,53.903,53.8895,53.80275,0.267850,0.367042,54.031190,53.80275,82.142857,75.714286
2020-03-31 12:50:00-04:00,51.600550,43.282514,53.891,53.8870,53.79975,0.248718,0.410825,54.024816,53.79975,19.354839,58.832565
2020-03-31 12:51:00-04:00,49.961651,41.039718,53.877,53.8730,53.79750,0.230952,0.416480,54.022011,53.79750,10.526316,37.341337


# Using a different Backtrader


In [28]:
#adding another backtrader
import bt as bt1

In [29]:
s = bt1.Strategy('s1', [bt1.algos.RunDaily(),
                       bt1.algos.SelectAll(),
                       bt1.algos.WeighEqually(),
                       bt1.algos.Rebalance()])

In [30]:
# create a backtest and run it
test = bt1.Backtest(s, df)
res = bt1.run(test)

s1
0% [############################# ] 100% | ETA: 00:00:00

In [31]:
res.display()

Stat                 s1
-------------------  -----------------------------------------------------------------------------------------------------------
Start                2020-03-30
End                  2020-04-02
Risk-free rate       0.00%

Total Return         563.95%
Daily Sharpe         11.97
Daily Sortino        100.92
CAGR                 1243973909555863860956502705020317563883606821000006340236036753068997056622084175066623591898666762240.00%
Max Drawdown         -74.21%
Calmar Ratio         16763217603531026644428967281203005161442492020733821726167057248795599335431683229428934273159659520.00

MTD                  92.52%
3m                   -
6m                   -
YTD                  563.95%
1Y                   -
3Y (ann.)            -
5Y (ann.)            -
10Y (ann.)           -
Since Incep. (ann.)  1243973909555863860956502705020317563883606821000006340236036753068997056622084175066623591898666762240.00%

Daily Sharpe         11.97
Daily Sortino        100.92
Daily 

In [32]:
# first let's see an equity curve
res.plot()

findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.


In [33]:
# ok and what about some stats?
res.stats

,s1
start,2020-03-30 12:13:00-04:00
end,2020-04-02 15:59:00-04:00
rf,0
total_return,5.63948
cagr,1.24397e+100
max_drawdown,-0.742085
calmar,1.67632e+100
mtd,0.925236
three_month,NaN
six_month,NaN


In [34]:
#store gains and losses
#return distribution look like?
res.plot_histogram()

C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\pandas\plotting\_core.py:2246: MatplotlibDeprecationWarning: 
The 'normed' kwarg was deprecated in Matplotlib 2.1 and will be removed in 3.1. Use 'density' instead.
  ax.hist(values, bins=bins, **kwds)


In [35]:
# and just to make sure everything went along as planned, let's plot the security weights over time
res.plot_security_weights()

## pyfolio

In [36]:
#https://www.backtrader.com/docu/analyzers/pyfolio/